In [1]:
import pandas as pd
import numpy as np
import scipy.io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
data = scipy.io.loadmat('/content/drive/MyDrive/ASSIGNMENT_2/data5.mat')
data = pd.DataFrame(data['x'])
print("shape of data is: ", data.shape)

shape of data is:  (2148, 73)


In [16]:
## shuffling the data
shuffled_data = data.sample(frac = 1, random_state = 7).reset_index(drop = True)
#shuffled_data.columns = list(range(data.shape[1]))
shuffled_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
0,9099.953116,34419.448400,12591.255480,18620.63712,2235.504601,21229.006820,41680.596380,169002.68050,138099.53810,57612.230960,34726.49806,39607.64698,5323.474615,13604.667440,5951.954708,7974.739527,2237.335039,8448.910441,23936.350210,133431.520600,155575.976700,107683.503600,61838.826250,29891.720930,5475.708579,27683.725520,9131.006312,14294.826350,378.942691,17040.588260,20001.756930,27563.001210,10706.739530,11551.853070,31962.063880,44190.886680,140.425265,1686.212907,1170.732378,619.270179,233.419546,1392.763022,1352.246639,949.346357,628.484465,761.633959,1049.815959,1049.514045,7.383080,4.258683,0.236529,0.086332,0.018593,0.001531,6.404204,5.475822,0.094963,0.008202,0.001600,0.000201,9.674465,2.166467,0.123968,0.018160,0.001644,0.000235,6.218616,3.728230,1.601582,0.303282,0.105261,0.020237,1.0
1,18545.804850,25605.492620,1668.275131,21654.12498,3677.349642,8999.087928,9256.006177,42351.82393,113792.73200,147831.651500,38208.38922,23759.13748,70.926836,325.233172,204.617607,146.981798,55.940062,246.419514,172.899231,290.023958,334.614121,246.616247,132.662535,93.254131,46.411854,72.470217,17.783717,55.164717,13.881040,33.591979,152.008318,126.268650,149.235183,153.398953,26.056361,21.665645,15.070310,30.545217,21.154777,17.599012,10.683676,21.879826,58.675889,113.320786,137.386084,71.483439,10.963912,3.127098,10.770977,1.030099,0.130368,0.042568,0.008751,0.001430,6.568659,3.478885,1.194281,0.529476,0.144836,0.046084,6.579068,2.341549,1.910509,0.632260,0.426809,0.050869,5.923631,3.121233,1.643689,0.701598,0.360398,0.140074,1.0
2,31221.136800,33857.898070,23264.267890,26745.16991,18819.132370,20747.693070,42013.266020,207522.64490,368593.94770,300048.260500,86212.35524,42020.56924,14774.811680,13097.833620,10953.788620,11195.764020,9589.574397,8334.849935,9219.566785,54437.925070,163425.753600,149318.424400,43064.687870,20234.164220,3369.003452,2891.060212,8776.026715,935.736450,5348.907707,4988.279668,1031.122915,13168.617970,26675.599890,22079.425240,4689.063298,2570.127057,260.571928,129.208638,354.222658,106.579161,275.271695,177.076403,138.148470,1646.457433,9879.343442,4921.108812,648.755982,267.029548,7.243279,4.441496,0.261800,0.033213,0.003865,0.001204,7.545151,4.363245,0.063260,0.011169,0.001451,0.000555,6.635780,4.910883,0.331001,0.083566,0.013873,0.006454,5.590924,3.257021,2.180618,0.471013,0.321156,0.098357,0.0
3,6045.378272,14044.801660,2527.740590,9413.66631,776.165541,6775.022914,8910.421889,41152.93675,40404.93494,30671.588110,19607.00581,11707.26269,4789.204676,7974.068580,605.009777,6229.019951,702.279665,3094.723793,11893.737390,18634.390280,8667.826605,8893.405255,15367.260330,11971.061250,5556.483432,6670.662274,1177.852511,5817.669696,1698.620222,2535.872109,869.842833,8430.694269,29986.633430,35603.419470,19560.924780,8704.744766,1803.954962,1209.437443,175.474510,1461.781488,687.649243,241.326257,528.775380,1144.655228,6010.970293,5662.994658,2758.658408,1105.024611,8.751960,2.599051,0.350896,0.222692,0.045891,0.011647,9.054824,2.375603,0.482899,0.062145,0.006351,0.002155,8.748672,2.653536,0.534616,0.042858,0.003393,0.001397,4.971711,4.153662,1.528452,0.991325,0.277555,0.029828,0.0
4,11386.552440,80965.113360,42101.232220,35655.30042,6504.796850,58693.062770,86640.185320,212875.78110,259075.16610,292543.567800,150141.64130,66682.74558,2013.494527,38374.776620,31966.448790,12200.450210,7404.332221,34655.950730,72994.163430,115603.551100,74015.263860,60713.515000,49672.417780,37582.076790,5611.244812,8596.638592,2105.744716,6581.176581,1713.984335,3948.193590,6790.586865,24190.739750,82749.580770,183452.744600,67427.367650,14172.090300,571.767668,2710.909555,950.328805,1403.394543,84.256924,1687.375244,2069.385398,963.264914,5683.638423,21392.870840,8698.302270,2097.234749,7.981432,3.68923

In [18]:
## splitting the data in input and labels
X = shuffled_data.iloc[:, :-1]
Y = shuffled_data.iloc[:, -1:].reset_index(drop=True)
print("shape of X is: ", X.shape)
print("shape of Y is: ", Y.shape)

shape of X is:  (2148, 72)
shape of Y is:  (2148, 1)


In [23]:
def split(X, Y, val_split = 0.1, test_split = 0.2):
  '''
  splits the input and output in
  train, validation and test set
  '''
  train_size = int(X.shape[0] * (1 - (val_split + test_split)))
  val_size = int(X.shape[0] * val_split)
  test_size = X.shape[0] - train_size - val_size                     

  X_tr = X.iloc[:train_size, :]
  Y_tr = Y.iloc[:train_size, :]

  X_val = X.iloc[train_size:train_size + val_size, :].reset_index(drop = True)
  Y_val = Y.iloc[train_size:train_size + val_size, :].reset_index(drop = True)

  X_ts = X.iloc[train_size + val_size:, :].reset_index(drop = True)
  Y_ts = Y.iloc[train_size + val_size:, :].reset_index(drop = True)

  return X_tr, Y_tr, X_val, Y_val, X_ts, Y_ts

In [24]:
X_train, y_train, X_val, y_val, X_test, y_test = split(X, Y, val_split=0.1, test_split = 0.2)
print("shape of X_train is: ", X_train.shape)
print("shape of y_train is: ",y_train.shape)
print("shape of X_val is: ",X_val.shape)
print("shape of y_val is: ",y_val.shape)
print("shape of X_test is: ",X_test.shape)
print("shape of y_test is: " , y_test.shape)

shape of X_train is:  (1503, 72)
shape of y_train is:  (1503, 1)
shape of X_val is:  (214, 72)
shape of y_val is:  (214, 1)
shape of X_test is:  (431, 72)
shape of y_test is:  (431, 1)


In [25]:
min_X_train_feat = list(X_train.min(axis=0))
max_X_train_feat = list(X_train.max(axis=0))

In [29]:
#### normalize data
def normalize(X, min_features, max_features):
  n = X.shape[1]
  m = X.shape[0]
  X1 = pd.DataFrame(np.zeros((m,n)))
  for i in range(n):
    X1.iloc[:,i] = (X.iloc[:,i] - min_features[i])/(max_features[i] - min_features[i])
  return X1

In [30]:
X_train_norm = normalize(X_train, min_X_train_feat, max_X_train_feat)
X_val_norm = normalize(X_val, min_X_train_feat, max_X_train_feat)
X_test_norm = normalize(X_test, min_X_train_feat, max_X_train_feat)
print("shape of X_train_norm is: ",X_train_norm.shape)
print("shape of X_val_norm is: ",X_val_norm.shape)
print("shape of X_test_norm is: ",X_test_norm.shape)

shape of X_train_norm is:  (1503, 72)
shape of X_val_norm is:  (214, 72)
shape of X_test_norm is:  (431, 72)


In [33]:
### converting dataframe to array
X_train_norm = np.array(X_train_norm)
y_train = np.array(y_train)

In [65]:
X_val_norm = np.array(X_val_norm)
y_val = np.array(y_val)
X_test_norm = np.array(X_test_norm)
y_test = np.array(y_test)

In [55]:
#### compute distance of data from all feature centres
def compute_distance(feature_centers,datapoint):
    return np.sum(np.power(datapoint-feature_centers,2),axis=1)

In [56]:
#### computing feature centres after some epochs
def kmeans(data,num_cluster_centers,epochs=1000)
    cluster = np.zeros((data.shape[0],1))
    center_indexes = np.random.random_integers(0,data.shape[0],num_cluster_centers)
    feature_centers = data[center_indexes]

    for epoch in range(epochs):
        distances = np.zeros((num_cluster_centers,1))
        for datapoint in range(data.shape[0]):
            distances = compute_distance(feature_centers,data[datapoint,:])
            cluster_index = np.argmin(distances)
            cluster[datapoint,0] = cluster_index

        for i in range(num_cluster_centers):
            cluster_points_indices = np.argwhere(cluster == i)
            cluster_points = data[cluster_points_indices[:,0]]
            if cluster_points.shape[0] != 0:
                feature_centers[i] = np.mean(cluster_points,axis=0)

    return feature_centers

In [59]:
centres = kmeans(X_train_norm, 100, epochs = 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: This function is deprecated. Please call randint(0, 1503 + 1) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [63]:
### H matrix for training data
H= np.zeros((X_train_norm.shape[0],100))
for i in range(X_train_norm.shape[0]):
    for j in range(100):
        H[i][j] = np.linalg.norm(X_train_norm[i]-centres[j])
H.shape

(1503, 100)

In [75]:
### H matrix for validation data
H_val = np.empty((X_val_norm.shape[0],100), dtype= float)
for i in range(X_val_norm.shape[0]):
    for j in range(100):
        H_val[i][j] = np.linalg.norm(X_val_norm[i]-centres[j])
H_val.shape

(214, 100)

In [76]:
### H matrix for test data
H_test = np.empty((X_test_norm.shape[0],100), dtype= float)
for i in range(X_test.shape[0]):
    for j in range(100):
        H_test[i][j] = np.linalg.norm(X_test_norm[i]-centres[j])
H_test.shape

(431, 100)

In [72]:
### calculating the weight matrix
W = np.dot(np.linalg.pinv(H), y_train)
W.shape

(100, 1)

In [80]:
### predicting validation set labels
Y_pred_val = np.dot(H_val, W)
for i in range(len(Y_pred_val)):
  if(Y_pred_val[i][0] >= 0.5):
    Y_pred_val[i][0] = 1
  else:
    Y_pred_val[i][0] = 0
Y_pred_val.shape

(214, 1)

In [81]:
#### function to calculate accuracy
def accuracy(Y_act, Y_pred):
  correct = 0
  incorrect = 0
  for i in range(len(Y_act)):
    if(Y_act[i][0] == 1):
      if(Y_pred[i][0] == 1):
        correct = correct + 1
      else:
        incorrect = incorrect + 1
    else:
      if(Y_pred[i][0]==1):
        incorrect = incorrect + 1
      else:
        correct = correct + 1
  acc = correct/(incorrect + correct)
  return acc

In [82]:
#### accuracy of validation set
acc_val = accuracy(y_val, Y_pred_val)
print(f"accuracy of validation set is {acc_val*100} %")

accuracy of validation set is 84.11214953271028 %


In [83]:
### predicting test set labels
Y_test_val = np.dot(H_test, W)
for i in range(len(Y_test_val)):
  if(Y_test_val[i][0] >= 0.5):
    Y_test_val[i][0] = 1
  else:
    Y_test_val[i][0] = 0
Y_test_val.shape

(431, 1)

In [85]:
### accuracy of test set
acc_test = accuracy(y_test, Y_test_val)
print(f"accuracy of validation set is {acc_test*100} %")

accuracy of validation set is 90.25522041763341 %
